# More Neural Network Testing

In [1]:
# Libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Tensorflow aliases
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.utils import plot_model

data = pd.read_csv('../parkinsons_updrs.csv')

# Drop the subject # column because it is useless
data = data.drop('subject#', axis=1)

X = data.drop(['motor_UPDRS', 'total_UPDRS'], axis=1)
y = data['total_UPDRS']

# Split into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Randomly select 10% of the training data to be the validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print(f"Original data: {len(X)}\nTraining set: {len(X_train)}\nValidation set: {len(X_val)}\nTesting set: {len(X_test)}")

# Weight regularization to keep the weights small
def build_model(neurons, layers, input_shape, activation='linear', regularizer=None, lam=0.01, dropout=False, drop_prob=0.2, summarize=False):
  model = Sequential()
  model.add(Flatten(input_shape=input_shape))
  if regularizer != None:
    for i in range(layers-1):
      model.add(Dense(neurons, activation=activation, kernel_regularizer=regularizer(lam)))
      if dropout:
        model.add(Dropout(0.2)) # randomly select nodes with probability 0.2
  else:
    model.add(Dense(neurons, input_shape=input_shape))
    for i in range(layers-1):
      model.add(Dense(neurons, activation=activation))
  

  model.add(Dense(1))
  model.compile(loss='mae', optimizer='adam', metrics=['mae'])

  if summarize:
    model.build()
    model.summary()
    
  return model

# Helper functions
def show_acc(h, e):
  plt.plot(range(e), h.history['mae'], label='Training')
  plt.plot(range(e), h.history['val_mae'], label='Validation')
  plt.ylim([0, np.max(h.history['mae'])+10])
  plt.legend()
  plt.show()

def build_and_plot(neurons, layers, input_shape, activation='linear', regularizer=None, lam=0.01, dropout=False, drop_prob=0.2, summarize=False, e=100, verbose=0):
  model = build_model(neurons, layers, input_shape, activation=activation, regularizer=regularizer, lam=lam, dropout=dropout, drop_prob=drop_prob, summarize=summarize)
  history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=e, verbose=verbose)
  show_acc(history, e)
  print("Max mae: ", np.max(history.history['mae']))
  print("Min mae: ", np.min(history.history['mae']))
  print("Test MAE: ", model.evaluate(X_test, y_test))

Original data: 5875
Training set: 3760
Validation set: 940
Testing set: 1175


In [3]:
# Model 32:
# build_and_plot(neurons=128, layers=4, input_shape=(X_train.shape[1],), activation='elu', dropout=False, e=1000, verbose=0)


# Model 45:
# build_and_plot(neurons=128, layers=4, input_shape=(X_train.shape[1],), activation='elu', dropout=True, drop_prob=0.4, e=1000, verbose=0)


# Model 44:
# build_and_plot(neurons=128, layers=4, input_shape=(X_train.shape[1],), activation='elu', dropout=True, drop_prob=0.3, e=1000, verbose=0)


# Model 35:
# build_and_plot(neurons=128, layers=4, input_shape=(X_train.shape[1],), regularizer=l2, activation='elu', dropout=False, e=1000, verbose=0)


# Model 31:
# build_and_plot(neurons=128, layers=4, input_shape=(X_train.shape[1],), activation='elu', dropout=False, e=500, verbose=0)